In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [2]:
## Read the ppdfs from the folder
loader=PyPDFDirectoryLoader("./us_census")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under \nthe Continuous Enrollment Provision.2 The American \nRescue Plan (ARP) enhanced Marketplace premium \nsubsidies for those with incomes above 400 percent \nof the poverty level as well as for unemployed people.3\nIn addition to national policies, individual states and \nthe District of Columbia can affect health insurance \ncoverage by making Marketplace or Medicai

In [3]:
len(final_documents)

316

In [4]:
## Embedding Using Huggingface
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

/Users/akhilbhardwaj/Desktop/Langchain/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-0.07903484 -0.01134113 -0.02312097  0.02844467  0.05053343  0.05317827
 -0.01907786  0.03456028 -0.10211373 -0.02915697  0.08524259  0.05650725
 -0.02545439 -0.0330849  -0.00635737  0.0409086  -0.00628109  0.00356744
 -0.03854131  0.03667686 -0.04289805  0.03425254 -0.03116894 -0.03793729
  0.01728391  0.0121492   0.00653119  0.01463566 -0.05529056 -0.15320705
  0.00730849  0.03202938 -0.04701122 -0.01595971  0.01874446  0.02642935
 -0.02306375  0.08438036  0.04182489  0.05278172 -0.03057599  0.01564261
 -0.01689071  0.00529408 -0.02417444  0.00412992 -0.01889935 -0.00150622
 -0.00836945 -0.03390067  0.03515956 -0.00553135  0.04910937  0.05971859
  0.05615963 -0.05105161  0.01475137 -0.01849961 -0.03284641  0.03576628
  0.04947709 -0.00938884 -0.26202121  0.09750335  0.01715687  0.04781389
 -0.00556319 -0.00298306 -0.02207355 -0.04463666 -0.05760488  0.04815875
 -0.05522211  0.01635334  0.03299244  0.02147079  0.01296214  0.01462307
  0.0217495  -0.00203     0.02099539  0.03353845 -0

In [6]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [7]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [8]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x30a148670> search_kwargs={'k': 3}


In [41]:
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

# Correct syntax for setting environment variables
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")




In [42]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the health insurance coverage?"
hf.invoke(query)

"What is the health insurance coverage?\n\nHealth insurance coverage is a type of insurance that helps pay for medical expenses, such as doctor visits, hospital stays, and prescription drugs. It can also help cover the cost of preventive care, such as vaccinations and screenings.\n\nThere are different types of health insurance coverage, including:\n\n1. Employer-sponsored health insurance: Many employers offer health insurance as a benefit to their employees. This type of coverage is usually provided through a group health plan, which is a plan that covers a group of people, such as employees of a company.\n2. Individual health insurance: If you don't have access to employer-sponsored health insurance, you can purchase individual health insurance on your own. This type of coverage is available through insurance marketplaces, such as the Health Insurance Marketplace, or directly from insurance companies.\n3. Medicare: Medicare is a federal health insurance program for people who are 65

In [43]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [44]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [45]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [48]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [52]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])



Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

comparison of ACS and CPS ASEC measures 
of health insurance coverage, refer to <www.
census.gov/topics/health/health-insurance/
guidance.html>.
9 Respondents may have more than one 
health insurance coverage type at the time 
of interview. As a result, adding the total 
number of people with private coverage and 
the total number with public coverage will 
sum to more than the total number with any 
coverage.
• From 2021 to 2022, nine states 
reported increases in private 
coverage, while seven reported 
decreases (Appendix Table B-2). 
DIFFERENCES IN THE 
UNINSURED RATE BY STATE 
IN 2022
In 2022, uninsured rates at the 
time of interview ranged across 
states from a low of 2.4 percent 
in Massachusetts to a high of 16.6 
percent in Texas, compared to the 
national rate of 8.0 percent.10 Ten 
of the 15 states with uninsured 
10 The uninsured rates in the Distri